In [1]:
import numpy as np
import pandas as pd
import eikon as ek

In [2]:
with open('key.txt','r') as f:
    key=f.readline()
    ek.set_app_key(key)

In [3]:
rics=['.IDCOT7TR','0#USBMK=']
usmt_rics=['US1MT','US2MT','US3MT','US6MT']
usyt_rics=['US1YT','US2YT','US3YT','US5YT','US7YT','US10YT','US30YT']
words=['US1MT','US2MT','US3MT','US6MT','US1YT','US2YT',
       'US3YT','US5YT','US7YT','US10YT','US30YT']

In [4]:
def name_to_digits(Name):
    ans=0
    for i in range(len(Name)):
        if Name[i]>='0' and Name[i]<='9':
            ans=ans*10+int(Name[i])
    return ans

In [5]:
def cal_mt(Name,Yield):
    Month=name_to_digits(Name)
    price=100/((1+Yield*0.01/12*Month)**(Month/12))
    return price

In [6]:
def cal_yt(Name,Coupon,Yield):
    Year=name_to_digits(Name)
    price=0
    for i in range(1,Year):
        price+=Coupon/(1+Yield*0.01)**i
    price+=(Coupon+100)/(1+Yield*0.01)**Year
    return price

In [7]:
data=pd.read_excel('value.xls')
a=list(data.iloc[:,1])
Coupon=[]
for i in a:
    t,_=i.split('/')
    Coupon.append(np.float64(t))
Coupon=Coupon[4:]
Coupon

[0.0, 1.5, 1.5, 1.5, 1.625, 1.625, 2.25]

In [23]:
ans=[]
for i in usmt_rics:
    Yield=ek.get_data(i+'=RR',fields=ek.TR_Field('TR.FiMaturityStandardYield'))
    Yield=np.float64(Yield[0].iloc[0,1])
    a=[]
    a.append(i)
    a.append(cal_mt(i,np.float64(Yield)))
    ans.append(a)

for i in range(len(usyt_rics)):
    Yield=ek.get_data(usyt_rics[i]+'=RR',fields=ek.TR_Field('TR.FiMaturityStandardYield'))
    Yield=np.float64(Yield[0].iloc[0,1])
    a=[]
    a.append(usyt_rics[i])
    a.append(cal_yt(usyt_rics[i],Coupon[i],np.float64(Yield)))
    ans.append(a)

ans

[['US1MT', 99.98809493982245],
 ['US2MT', 99.95236881207413],
 ['US3MT', 99.89499821428255],
 ['US6MT', 99.5823320957744],
 ['US1YT', 98.40526494146425],
 ['US2YT', 99.93696055108556],
 ['US3YT', 100.02156797540934],
 ['US5YT', 100.1097827762254],
 ['US7YT', 100.37533154966717],
 ['US10YT', 99.71647896673026],
 ['US30YT', 102.15733369529]]

In [25]:
estimate=pd.DataFrame(ans,columns=['Name','Value'])
estimate.to_excel('estimate.xls',index=False)